# Processing Invoices Using Box MCP Server

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import json
import sqlite3
import warnings
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)


True

In [ ]:
# Load API key and model
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL_NAME = "gpt-4"

In [13]:
# Initialize GPT client
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Local invoice folder
LOCAL_INVOICE_FOLDER = "../data/invoices"

# Helper: Parse JSON from GPT response

In [ ]:
from PyPDF2 import PdfReader
import re

def parse_json(content):
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        match = re.search(r"\{.*\}", content, re.DOTALL)
        if match:
            try:
                return json.loads(match.group())
            except json.JSONDecodeError:
                pass
    return {"error": "Failed to parse JSON", "raw": content}

# GPT-based invoice extractor

In [9]:

def extract_invoice_fields(file_path: str, file_name: str):
    print(f"Processing invoice: {file_name}")
    
    # Extract text from PDF
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""

    # GPT prompt
    prompt = (
        "Extract the following fields from this invoice:\n"
        "1. client_name (string)\n"
        "2. invoice_amount (float)\n"
        "3. product_name (string)\n"
        "Return result as a JSON object.\n\n"
        f"Invoice text:\n{text}"
    )

    # GPT call
    response = openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )

    result = parse_json(response.choices[0].message.content)
    result["file"] = file_name
    print(json.dumps(result, indent=4))
    return result

# SQLite Database Setup + Insert

In [17]:
# Connect to SQLite database
os.makedirs("db", exist_ok=True)
connection = sqlite3.connect("db/invoices.db")
cursor = connection.cursor()

# Create table if not exists
cursor.execute("""
    CREATE TABLE IF NOT EXISTS invoices (
        file TEXT PRIMARY KEY UNIQUE,
        client TEXT,
        amount REAL,
        product TEXT
    )
""")
connection.commit()

# Insert or update invoice data
def save_to_db(data: dict):
    product_str = (
        ", ".join(data["product_name"]) if isinstance(data["product_name"], list)
        else str(data["product_name"])
    )
    cursor.execute(
        """
        INSERT INTO invoices (file, client, amount, product)
        VALUES (?, ?, ?, ?)
        ON CONFLICT(file) DO UPDATE SET
            client=excluded.client,
            amount=excluded.amount,
            product=excluded.product
        """,
        (
            data["file"],
            data["client_name"],
            data["invoice_amount"],
            product_str,  # ✅ This is the fixed part
        ),
    )
    connection.commit()


# Loop Through Invoices and Save to DB

In [18]:
# Loop through all PDF invoices
for file_name in os.listdir(LOCAL_INVOICE_FOLDER):
    if file_name.endswith(".pdf"):
        file_path = os.path.join(LOCAL_INVOICE_FOLDER, file_name)
        try:
            # Extract fields using GPT
            data = extract_invoice_fields(file_path, file_name)

            # Save to database
            save_to_db(data)

        except Exception as e:
            print(f"Failed to process {file_name}: {e}")


Processing invoice: demo-invoice-20tax-2.pdf
{
    "client_name": "ACME Inc",
    "invoice_amount": 577271.99,
    "product_name": [
        "SpeakerCable",
        "SurroundSoundReceive",
        "TelevisionM12037",
        "Soundbar",
        "0.1 Cable"
    ],
    "file": "demo-invoice-20tax-2.pdf"
}
Processing invoice: demo-invoice-20tax-9.pdf
{
    "client_name": "ACME Inc",
    "invoice_amount": 1327.27,
    "product_name": [
        "Polyol",
        "Diisocyanate",
        "Carbon Dioxide",
        "Laser",
        "Lens",
        "Oleic Acid",
        "Glycerine",
        "Sodium Tallowate",
        "Paint Base",
        "Polypropylene",
        "Rubber",
        "Additive",
        "Pigment",
        "Aluminum Silicate",
        "Magnesium Silicate",
        "Zinc Oxide",
        "Distilled Solvent",
        "Petroleum Distillate",
        "Sulfur Dioxide",
        "Sodium Benzoate",
        "Dust cap",
        "Ferrite cap",
        "Cone and coil assembly",
        "Cleaner